In [2]:
library(nichenetr)
library(tidyverse)

In [3]:
hnscc_expression = readRDS("./data/hnscc_expression.rds")
expression = hnscc_expression$expression
sample_info = hnscc_expression$sample_info # contains meta-information about the cells

In [4]:
tumors_remove = c("HN10","HN","HN12", "HN13", "HN24", "HN7", "HN8","HN23")

CAF_ids = sample_info %>% filter(`Lymph node` == 0 & !(tumor %in% tumors_remove) & `non-cancer cell type` == "CAF") %>% pull(cell)
malignant_ids = sample_info %>% filter(`Lymph node` == 0 & !(tumor %in% tumors_remove) & `classified  as cancer cell` == 1) %>% pull(cell)

expressed_genes_CAFs = expression[CAF_ids,] %>% apply(2,function(x){10*(2**x - 1)}) %>% apply(2,function(x){log2(mean(x) + 1)}) %>% .[. >= 4] %>% names()
expressed_genes_malignant = expression[malignant_ids,] %>% apply(2,function(x){10*(2**x - 1)}) %>% apply(2,function(x){log2(mean(x) + 1)}) %>% .[. >= 4] %>% names()


In [6]:
ligand_target_matrix = readRDS("./data/ligand_target_matrix.rds")
pemt_geneset = readr::read_tsv("./data/pemt_signature.txt", col_names = "gene") %>% pull(gene) %>% .[. %in% rownames(ligand_target_matrix)]
background_expressed_genes = expressed_genes_malignant %>% .[. %in% rownames(ligand_target_matrix)]


Rows: 100 Columns: 1
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (1): gene

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [7]:
lr_network = readRDS("./data/lr_network.rds")

ligands = lr_network %>% pull(from) %>% unique()
expressed_ligands = intersect(ligands,expressed_genes_CAFs)

receptors = lr_network %>% pull(to) %>% unique()
expressed_receptors = intersect(receptors,expressed_genes_malignant)

potential_ligands = lr_network %>% filter(from %in% expressed_ligands & to %in% expressed_receptors) %>% pull(from) %>% unique()
head(potential_ligands)

[1] "HGF"     "TNFSF10" "TGFB2"   "TGFB3"   "INHBA"   "CD99"

In [9]:
ligand_activities = predict_ligand_activities(geneset = pemt_geneset, background_expressed_genes = background_expressed_genes, ligand_target_matrix = ligand_target_matrix, potential_ligands = potential_ligands)
best_upstream_ligands = ligand_activities %>% top_n(20, pearson) %>% arrange(-pearson) %>% pull(test_ligand)


利用top-ranked ligands预测基因是否属于p-EMT，并评估分类效果

In [11]:
k = 3 # 3-fold
n = 2 # 2 rounds

pemt_gene_predictions_top20_list = seq(n) %>% lapply(assess_rf_class_probabilities, folds = k, geneset = pemt_geneset, background_expressed_genes = background_expressed_genes, ligands_oi = best_upstream_ligands, ligand_target_matrix = ligand_target_matrix)


In [12]:
# get performance: auroc-aupr-pearson
target_prediction_performances_cv = pemt_gene_predictions_top20_list %>% lapply(classification_evaluation_continuous_pred_wrapper) %>% bind_rows() %>% mutate(round=seq(1:nrow(.)))

In [13]:
target_prediction_performances_cv$auroc %>% mean()
target_prediction_performances_cv$aupr %>% mean()
target_prediction_performances_cv$pearson %>% mean()

[1] 0.7318148

[1] 0.08455258

[1] 0.1751712

评估属于感兴趣的基因集是否被top-predicted, 此处利用前5% predicted target

In [16]:
target_prediction_performances_discrete_cv = pemt_gene_predictions_top20_list %>% lapply(calculate_fraction_top_predicted, quantile_cutoff = 0.95) %>% bind_rows() %>% ungroup() %>% mutate(round=rep(1:length(pemt_gene_predictions_top20_list), each = 2))

In [ ]:
# 属于前5%的p-EMT gene
target_prediction_performances_discrete_cv %>% filter(true_target) %>% .$fraction_positive_predicted %>% mean()
# 属于前5%的非p-EMT gene
target_prediction_performances_discrete_cv %>% filter(!true_target) %>% .$fraction_positive_predicted %>% mean()

In [17]:
# p-EMT是否在前5%富集
target_prediction_performances_discrete_fisher = pemt_gene_predictions_top20_list %>% lapply(calculate_fraction_top_predicted_fisher, quantile_cutoff = 0.95) 
target_prediction_performances_discrete_fisher %>% unlist() %>% mean()

[1] 8.386529e-11

In [18]:
# 那个p-EMT基因在CV的每一轮中都可以良好预测
top_predicted_genes = seq(length(pemt_gene_predictions_top20_list)) %>% lapply(get_top_predicted_genes,pemt_gene_predictions_top20_list) %>% reduce(full_join, by = c("gene","true_target"))
top_predicted_genes %>% filter(true_target)

gene,true_target,predicted_top_target_round1,predicted_top_target_round2
<chr>,<lgl>,<lgl>,<lgl>
MMP1,TRUE,TRUE,TRUE
COL1A1,TRUE,TRUE,TRUE
F3,TRUE,TRUE,TRUE
MT2A,TRUE,TRUE,TRUE
PLAU,TRUE,TRUE,TRUE
MMP2,TRUE,TRUE,TRUE
IGFBP3,TRUE,TRUE,TRUE
MMP10,TRUE,TRUE,TRUE
TNC,TRUE,TRUE,TRUE
